# [해커톤3] 구글 리뷰 크롤링하기

> * urllib는 파이썬에서 인터넷에서 데이터를 받아 오는 기능들이 있는 패키지로 기본적으로 내장되어 있음
> * urllib에 인터넷 주소URL을 넣고 실행하면 데이터를 텍스트 형태로 받아올 수 있음 : 크롤링
> * BeautifulSoup은 데이터를 추출하는데 필요한 기능이 들어 있는 라이브러리
> * BeautifulSoup은 크롤링한 데이터에서 필요한 내용만 추출하는 라이브러리 : 파싱

## 1. 크롤링 웹페이지 불러오기

In [1]:
import urllib
import urllib.request
from urllib.request import urlopen
import selenium 
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd
import os

In [2]:
link = 'https://www.google.com/maps/place/%EB%BF%8C%EB%A6%AC%EA%B3%B5%EC%9B%90/@36.2854269,127.3858948,17z/data=!4m7!3m6!1s0x3565496076e32ced:0x4f8951c6d2211063!8m2!3d36.2854269!4d127.3880835!9m1!1b1?hl=ko'
driver = webdriver.Chrome('./chromedriver')
driver.get(link)                 # 자동으로 해당 링크를 연다
os.makedirs('result', exist_ok = True)   # 결과 저장 디렉토리

### 1.1 자동 스크롤 다운 하기

스크래핑하기 전에 대부분의 다른 최신 웹 사이트와 마찬가지로 페이지가 AJAX(동적페이지)를 사용하여 구현되므로 모든 리뷰를 아래로 스크롤하여 모든 리뷰를 로드해야한다.

In [3]:
# 정렬(최신순)-스크롤다운-loadmore

scroll_cnt = 3

for i in range(scroll_cnt):
    # scroll to bottom :
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)
   
    # click 'load more':
    try:
        load_more = driver.find_elements_by_class_name("ODSEW-KoToPc-ShBeI gXqMYb-hSRGPd")
        for i in load_more:
            i.click()
    except:
         print("can't find load more button...")

### 1.2 리뷰 정보 가져오기

In [4]:
# # 리뷰 가져오기01 
# elem = driver.find_element_by_class_name('widget-pane-content-holder')
# reviews = elem.find_elements_by_xpath('//*[@class="section-review NIyLF-haAclf gm2-body-2"]//div[@class="ODSEW-ShBeI-content"]')
# print('there are %d reviews avaliable!' % len(reviews))
# print('writing the data...')

In [5]:
# 리뷰 가져오기02 : 최대 930건으로 제한이 있음
elem = driver.find_element_by_class_name('widget-pane-content-holder')
reviews = elem.find_elements_by_xpath('//*[@class="section-layout"]//div[@class="ODSEW-ShBeI-content"]')
print('there are %d reviews avaliable!' % len(reviews))
print('writing the data...')

there are 930 reviews avaliable!
writing the data...


## 2. 리뷰 데이터셋으로 저장

In [6]:
# 데이터셋 생성

df = pd.DataFrame(columns=['name', 'ratings', 'date', 'comment'])

In [7]:
# 해당 정보 데이터셋에 저장

for review in reviews:
    soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')
    
    name = soup.find(class_= "ODSEW-ShBeI-title").text
    
    ratings = int(soup.find('span', class_='ODSEW-ShBeI-H1e3jb').get('aria-label').replace('별표', '').replace('개','').strip())
    
    date= soup.find(class_= "ODSEW-ShBeI-RgZmSc-date").text
     
    comment = soup.find('span', class_='ODSEW-ShBeI-text').text
    if not comment:
        comment = "없음" 
        
    df = df.append({
        'name': name,
        'ratings': ratings,
        'date' : date,
        'comment' : comment
    }, ignore_index=True)


In [8]:
# csv 파일로 저장

filename = datetime.now().strftime('result/%Y-%m-%d_%H-%M-%S.csv')   
df.to_csv(filename, encoding='utf-8-sig', index=False)
driver.stop_client()       
driver.close()         

print('저장완료')

저장완료


## 3. 저장 결과 확인

In [9]:
import os
from os.path import join

In [10]:
data_dir = os.getenv('HOME')+'/result'

place_path = join(data_dir, '2021-06-20_19-36-33.csv')
place_check = pd.read_csv(place_path)

place_check.head(20)

,name,ratings,date,comment
0,너구리먹고싶당,5,2시간 전,ㅇㅇㅇ
1,박미정,5,1일 전,조용해서좋고 나의 뿌리를 알수있어서 좋았어요
2,김인호,5,1일 전,없음
3,이권환 도진道眞 TV,5,1일 전,"""조상은 자손의 뿌리요,\n자손은 조상의 숨구멍이다.""\n\n- 甑山道 安雲山 太上..."
4,카르페,4,3일 전,없음
5,민기짱짱,5,3일 전,뿌리공원에는 많은 사람들이 산책을 하고 있어요.뿌리공원에는 볼거리가 많아요.뿌리공원...
6,김동섭,4,4일 전,없음
7,김지예,5,6일 전,없음
8,Jinwoo Yang (Alex),5,1주 전,언제가도 좋은 곳...마음이 편해지는 힐링 스팟입니다.
9,HN Y,5,1주 전,없음


### 참고 자료 출처 :
* http://egloos.zum.com/mcchae/v/11281390
* 김경록서영덕, 2018, "한입에 웹 크롤링", 비제이퍼블릭.

### 이슈사항 :

1. Google Map review는 최대 930건만 크롤링 가능
2. 찾고자 하는 해당 정보의 HTML 태그가 주기적으로 변화  

### [참고] 요소 찾기

id_r = r.find('button', 
              class_='section-review-action-menu')['data-review-id']
username = r.find('div', 
                  class_='section-review-title').find('span').text
try:
    review_text = r.find('span', class_='section-review-text').text
except Exception:
    review_text = None
rating = r.find('span', class_='section-review-stars')['aria-label']
rel_date = r.find('span', class_='section-review-publish-date').text